In [2]:
using Graphs, GraphIO, FileIO
using CSV, DataFrames

using GMT

include("./src/cubes.jl")
include("./src/network.jl")

add_properties (generic function with 1 method)

In [4]:
## Get Vrancea subregion

# Read data
path = "./data/"
region = "Vrancea"
filepath = path * region * ".csv"

df = CSV.read(filepath, DataFrame);
minlon_vrancea = minimum(df.Longitude)
maxlon_vrancea = maximum(df.Longitude)
minlat_vrancea = minimum(df.Latitude)
maxlat_vrancea = maximum(df.Latitude)

46.0

In [5]:
println(minlon_vrancea, maxlon_vrancea, minlat_vrancea,maxlat_vrancea  )

26.027.045.246.0


In [123]:
# ## Get Vrancea subregion

# # Read data
# path = "./data/"
# region = "Vrancea"
# filepath = path * region * ".csv"

# df = CSV.read(filepath, DataFrame);
# minlon_vrancea = minimum(df.Longitude)
# maxlon_vrancea = maximum(df.Longitude)
# minlat_vrancea = minimum(df.Latitude)
# maxlat_vrancea = maximum(df.Latitude)

# # Read data
# path = "./data/"
# region = "Romania"
# filepath = path * region * ".csv"

# mkpath("./gmt/$region")

# df = CSV.read(filepath, DataFrame);

# cell_size = 5.0
# minimum_magnitude = 2

# # graph_path = "./networks/$region/cell_size_$(cell_size)km/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).txt"

# region = "Vrancea"
# df = df[(df.Longitude .> minlon_vrancea) .& (df.Longitude .< maxlon_vrancea) .& 
#         (df.Latitude .> minlat_vrancea) .& (df.Latitude .< maxlat_vrancea), :]

# df_filtered = df[df.Magnitude .> minimum_magnitude,:] 

# # Split into cubes
# df_filtered, df_filtered_cubes = region_cube_split(df_filtered,cell_size=cell_size,energyRelease=true);

# # Create network
# MG = create_network(df_filtered, df_filtered_cubes)
# connectivity = degree(MG);

# # edgelist_array = Matrix(edgelist);
# edgelist = collect(edges(MG)) |> DataFrame;

In [185]:
# Read data
path = "./data/"
region = "California"
filepath = path * region * ".csv"

mkpath("./gmt/$region")

df = CSV.read(filepath, DataFrame);

cell_size = 5.0
minimum_magnitude = 4

# graph_path = "./networks/$region/cell_size_$(cell_size)km/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).txt"

df_filtered = df[df.Magnitude .> minimum_magnitude,:] 

# Split into cubes
df_filtered, df_filtered_cubes = region_cube_split(df_filtered,cell_size=cell_size,energyRelease=true);
# Create network
MG = create_network(df_filtered, df_filtered_cubes)
connectivity = degree(MG);
# edgelist_array = Matrix(edgelist);
edgelist = collect(edges(MG)) |> DataFrame;

In [187]:
# Get region's coordinates
min_lon = minimum(df_filtered_cubes.cubeLongitude) -0.1
max_lon = maximum(df_filtered_cubes.cubeLongitude) +0.1
min_lat = minimum(df_filtered_cubes.cubeLatitude) -0.1
max_lat = maximum(df_filtered_cubes.cubeLatitude) +0.1
min_dep = minimum(df_filtered_cubes.cubeDepth);
max_dep = maximum(df_filtered_cubes.cubeDepth);

# Create the map coordinates
map_coords = [min_lon,max_lon,min_lat,max_lat]
map_coords_depth = [min_lon,max_lon,min_lat,max_lat,-max_dep,-min_dep]

# Colormap for the region topography
C_map = makecpt(cmap=:geo, range=(-8000,8000), continuous=true);
# Relief map of the region
relief_map = grdcut("@earth_relief_30s", region=map_coords);

makecpt [WARNING]: Without inc in -T option, -Z has no effect (ignored)
grdcut [WARNING]: (w - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: w reset from 6.6237 to 6.61666666667
grdcut [WARNING]: (e - x_min) must equal (NX + eps) * x_inc), where NX is an integer and |eps| <= 0.0001.
grdcut [WARNING]: e reset from 18.4409 to 18.4416666667
grdcut [WARNING]: (s - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: s reset from 35.5922 to 35.5916666667
grdcut [WARNING]: (n - y_min) must equal (NY + eps) * y_inc), where NY is an integer and |eps| <= 0.0001.
grdcut [WARNING]: n reset from 47.0204 to 47.025


In [188]:
using Geodesy

map_coords_depth

x0_lla = LLA(min_lat,min_lon,-min_dep)
xf_lla = LLA(min_lat,max_lon,-min_dep)
lon_dist_in_km = Geodesy.euclidean_distance(xf_lla,x0_lla) / 1000

1069.043191122469

In [189]:
z_ratio = 200 * max_dep / lon_dist_in_km

12.020094329872508

In [192]:
# marker_size = connectivity ./10

# control marker color by connectivity
C_markers = makecpt(cmap=:seis, range=(minimum(connectivity),maximum(connectivity)), inverse=true);
zcolor_control = connectivity;

perspective = (175,20)
quality = 100

100

In [198]:
# Basemap to define the axes
basemap(limits=map_coords_depth, proj=:merc, zsize=z_ratio, frame="SEnwZ1+b xafg yafg zaf+lDepth(km)", 
        par=(FONT_LABEL="24p,Palatino-Roman",MAP_LABEL_OFFSET=0.8, FONT_ANNOT="20p,Palatino-Roman,black"), view=perspective)



# Edges, plotted manually
for i in range(1,nrow(edgelist))
    line_coords = DataFrame(lats = [df_filtered_cubes.cubeLatitude[edgelist.src[i]],df_filtered_cubes.cubeLatitude[edgelist.dst[i]]],
                    lons =[df_filtered_cubes.cubeLongitude[edgelist.src[i]],df_filtered_cubes.cubeLongitude[edgelist.dst[i]]],
                    deps= [df_filtered_cubes.cubeDepth[edgelist.src[i]],df_filtered_cubes.cubeDepth[edgelist.dst[i]]])

    plot3d!(line_coords.lons, line_coords.lats, -line_coords.deps, JZ=string(z_ratio) * "c", proj=:merc, pen=(:thinner,:black), alpha=50, view=perspective)
end

# Nodes
scatter3!(df_filtered_cubes.cubeLongitude, df_filtered_cubes.cubeLatitude, -df_filtered_cubes.cubeDepth,
limits=map_coords_depth,frame="SEnwZ1+b",proj=:merc, marker=:cube,markersize=0.3, 
cmap=C_markers, zcolor=connectivity, 
alpha=50, view=perspective)

# Colorbar
colorbar!(limits=map_coords, pos=(paper=(19.0,1.5), size=(8,0.5)), shade=0.4, xaxis=(annot=4,), frame=(xlabel="Degree",),
        par=(FONT_LABEL="24p,Palatino-Roman,black",MAP_LABEL_OFFSET=0.6,FONT_ANNOT="20p,Palatino-Roman,black"),view=(180,90))



# Relief map
grdview!(relief_map, proj=:merc, surftype=(image=quality,), 
cmap=C_map, zsize=0.1, alpha=25 ,yshift=z_ratio - 0.4, view=perspective,
)


legend!((
        label=(txt="L=$(cell_size) , M"*subscript(min)*"=$(minimum_magnitude) ", justify=:L, font=(14, "Palatino-Roman")),
        ),
        limits=map_coords, pos=(paper=(-2.3,6.4), width=3.7, justify=:BL, spacing=2.4),
        clearance=(0.1,0.1), box=(pen=0.1, fill=:azure1),
        figsize=10, proj=:Mercator, view = (180,90),
        savefig="./gmt/$region/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude)_test.png",)


In [101]:
# using DelimitedFiles

## Motifs plot polygons

In [102]:
# motif="Triangle"
# network_target_path ="./networks/$region/cell_size_$(cell_size)km/"
# motif_filename = "motif$(motif)_$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).csv"

# motifs = readdlm(network_target_path * motif_filename, ',', Int64);

# # Basemap to define the axes
# basemap(limits=map_coords_depth, proj=:merc, zsize=6, frame="SEnwZ1+b xafg yafg zaf+lDepth(km)", view=(135,20))

# # Edges, plotted manually
# for i in range(1,nrow(edgelist))
#     line = DataFrame(lats = [df_filtered_cubes.cubeLatitude[edgelist.src[i]],df_filtered_cubes.cubeLatitude[edgelist.dst[i]]],
#                     lons =[df_filtered_cubes.cubeLongitude[edgelist.src[i]],df_filtered_cubes.cubeLongitude[edgelist.dst[i]]],
#                     deps= [df_filtered_cubes.cubeDepth[edgelist.src[i]],df_filtered_cubes.cubeDepth[edgelist.dst[i]]])

#     plot3d!(line.lons,line.lats,-line.deps, JZ="6c", proj=:merc, pen=(:thinner,:black), alpha=50, view=(135,20))
# end

# # Nodes
# scatter3!(df_filtered_cubes.cubeLongitude, df_filtered_cubes.cubeLatitude, -df_filtered_cubes.cubeDepth,
# limits=map_coords_depth, frame="SEnwZ1+b xafg yafg zaf",proj=:merc, marker=:cube,markersize=0.1, #markersize=marker_size,
# cmap=C_markers, zcolor=connectivity, 
# alpha=50, view=(135,20))

# # Motifs
# for i in range(1,size(motifs,1))
#     motif = DataFrame(lats = [df_filtered_cubes.cubeLatitude[motifs[i,1]], df_filtered_cubes.cubeLatitude[motifs[i,2]], df_filtered_cubes.cubeLatitude[motifs[i,3]]],
#                     lons =[df_filtered_cubes.cubeLongitude[motifs[i,1]], df_filtered_cubes.cubeLongitude[motifs[i,2]], df_filtered_cubes.cubeLongitude[motifs[i,3]]],
#                     deps= [df_filtered_cubes.cubeDepth[motifs[i,1]], df_filtered_cubes.cubeDepth[motifs[i,2]], df_filtered_cubes.cubeDepth[motifs[i,3]]])
#     plot3d!(motif.lons,motif.lats,-motif.deps, JZ="6c", proj=:merc, L=true, G=:red, alpha=50, view=(135,20))
# end

# # Colorbar
# colorbar!(pos=(outcell_size=:BC, offset=(0,1.5)), shade=0.4, JZ="6c", xaxis=(annot=:auto,), frame=(xlabel="Degree",),par=(MAP_LABEL_OFFSET=0.8,))

# # Relief map
# grdview!(relief_map, proj=:merc, surftype=(image=2000,), 
# cmap=C_map, zsize=0.05, alpha=10 ,yshift=5.6, view=(135,20),
# savefig="./networks/$region/cell_size_$(cell_size)km/$(region)_cell_size_$(cell_size)km_minmag_$(minimum_magnitude).png", 
# show=true)